In [2]:
import tensorflow as tf
from wrapper.model_wrapper import ModelWrapper
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [2]:
from datetime import datetime
import json
import mlflow
from mlflow.models.signature import infer_signature

TRACKING_URI = 'http://localhost:5000'
EXPERIMENT_ID = 326755177053281989

wrapper_model = ModelWrapper()
mlflow.set_tracking_uri(TRACKING_URI)
run_tag = {'model':'clothing_classification','framework':'tensorflow'}
now = datetime.now()
run_name = f"run_name_{now.strftime('%m%d%Y%H%M%S')}"
with mlflow.start_run(experiment_id=EXPERIMENT_ID,tags=run_tag) as run_mlflow:
    mlflow.set_tag("mlflow.runName", run_name)
    # train model
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
            ])
    model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=10)
    # evaluate
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    # log metric
    metrics = {"test_loss":test_loss,
               "test_accuracy":test_acc
              }
    mlflow.log_metrics(metrics)
    # log model summary as a text file
    model_summary_path = 'model_summary.txt'
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)
    mlflow.log_text(model_summary,model_summary_path)
    # save classname file
    class_names_path = "class_names.txt"
    with open(class_names_path, "w") as fp:
        json.dump(class_names, fp)
    # save file model
    model_path = 'save_model'
    model.save(model_path)
    # input-output signature
    signature = infer_signature(train_images)
    # serve model
    artifacts = {"class_names_path": class_names_path,'artifact_path':model_path}
    mlflow.pyfunc.log_model(
        artifact_path='model',
        artifacts=artifacts,
        code_path=['wrapper'],
        python_model=ModelWrapper(),
        signature=signature
    )

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 3.3019 - accuracy: 0.6715
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6863 - accuracy: 0.7304
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5898 - accuracy: 0.7849
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5340 - accuracy: 0.8108
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5172 - accuracy: 0.8166
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5123 - accuracy: 0.8205
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4967 - accuracy: 0.8262
Epoch 8/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4799 - accuracy: 0.8338
Epoch 9/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4819 - accuracy: 0.8343
Epoch 10/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.487

c:\Users\janebhop.saw\Anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
#predict 
logged_model = 'runs:/a9269d0619114047ae5dfa21029268b2/model'
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model.predict(test_images[0:5])

1/1 [==============================] - 0s 77ms/step


[('Ankle boot', 0.98597986),
 ('Pullover', 0.86687165),
 ('Trouser', 1.0),
 ('Trouser', 1.0),
 ('Shirt', 0.87008446)]